# Language Translation Model 

This notebook attempts to perform an effective language translation from Hindi to English language<br/>
The dataset being used in this project is the IIT Bombay Hindi English Corpus which has been provided for free use on the internet 

## Step 1: Importing the Dataset

First we would start with the imports 

In [91]:
import numpy as np 
import tensorflow as tf
import re 

We now go ahead and import the required datatset

In [92]:
from datasets import load_dataset
corpus_data = load_dataset('cfilt/iitb-english-hindi')
print(corpus_data)

Using custom data configuration cfilt--iitb-english-hindi-911387c6837f8b91
Reusing dataset parquet (C:\Users\parth\.cache\huggingface\datasets\cfilt___parquet\cfilt--iitb-english-hindi-911387c6837f8b91\0.0.0\7328ef7ee03eaf3f86ae40594d46a1cec86161704e02dd19f232d81eee72ade8)
100%|██████████| 3/3 [00:00<00:00, 14.43it/s]

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 1659083
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 2507
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 520
    })
})


We see that the corpus has the train, test and validation sets prepared by default<br/>
We can now go ahead and store them individually for further processing 

In [93]:
train_data = corpus_data["train"]["translation"]
test_data = corpus_data["test"]["translation"]
validation_data = corpus_data["validation"]["translation"]

In [94]:
train_data[0]

{'en': 'Give your application an accessibility workout',
 'hi': 'अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें'}

In [95]:
train_data_en = [train_datum['en'] for train_datum in train_data]
train_data_hi = [train_datum['hi'] for train_datum in train_data]

test_data_en = [test_datum['en'] for test_datum in test_data]
test_data_hi = [test_datum['hi'] for test_datum in test_data]

validation_data_en = [validation_datum['en'] for validation_datum in validation_data]
validation_data_hi = [validation_datum['hi'] for validation_datum in validation_data]

## Step 2: Text preprocessing on the input data

We first try to perform preprocessing on both English and Hindi sentences.<br/>
This includes
<ul>
    <li>Removing sentences whose length is more than a defined value</li>
    <li>Removing unwanted characters from the remaining sentences</li>
    <li>Converting the sentences to a sequence of numbers (embeddings) so that they can be fed to the model</li>
    <li>Padding the sequences so that they are all of uniform length</li>
</ul>

We first remove unwanted characters like URL characters, new line characters, single quotes, numbers etc. from the text

In [97]:
def purge_unwanted_characters(data):
    """To remove the unwanted characters from the input data"""

    #Removing URLs with a regular expression
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    data = url_pattern.sub(r'', data)
    
    # Remove Emails
    data = re.sub(r'\S*@\S*\s?', '', data)
    
    # Remove new line characters
    data = re.sub(r'\s+', ' ', data)

    # Remove distracting single quotes
    data = re.sub(r"\'", "", data)

    # Remove numbers from text 
    data = re.sub(r'\d', '', data)

    # Remove underscores and other special characters from text 
    data = re.sub(r'[_#$%]', '', data)
        
    return data

string = "www.example.com 'is' the number 1_ website on this planet"
purge_unwanted_characters(string)

' is the number  website on this planet'

In [98]:
def preprocess_english(data):
    processed_data = []
    for sentence in data: 
        processed_data.append(purge_unwanted_characters(sentence).lower())
    return processed_data

train_data_en = preprocess_english(train_data_en)
test_data_en = preprocess_english(test_data_en)
validation_data_en = preprocess_english(validation_data_en)

Now, we will proceed onto preprocessing of Hindi sentences. Note that here we will be trying to not only remove the unwanted characters as mentioned earlier, we will also try to remove english characters as they won't help the model in training. 

In [99]:
def preprocess_hindi(data):
    processed_data = []
    for sentence in data:
        processed_sentence = purge_unwanted_characters(sentence)
        processed_sentence.replace("।", '') # remove the hindi full stop (purn viram)
        processed_data.append(re.sub(r'[a-zA-Z]', '', processed_sentence))
    return processed_data

train_data_hi = preprocess_hindi(train_data_hi)
test_data_hi = preprocess_hindi(test_data_hi)
validation_data_hi = preprocess_hindi(validation_data_hi)

In [100]:
in_train = train_data_en
out_train = train_data_hi 

In [101]:
for i in range(5):
    print(in_train[i], out_train[i])

give your application an accessibility workout अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें
accerciser accessibility explorer एक्सेर्साइसर पहुंचनीयता अन्वेषक
the default plugin layout for the bottom panel निचले पटल के लिए डिफोल्ट प्लग-इन खाका
the default plugin layout for the top panel ऊपरी पटल के लिए डिफोल्ट प्लग-इन खाका
a list of plugins that are disabled by default उन प्लग-इनों की सूची जिन्हें डिफोल्ट रूप से निष्क्रिय किया गया है


In [102]:
en_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='', oov_token='<OOV>', lower=False)
hi_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='', oov_token='<OOV>', lower=False)

en_tokenizer.fit_on_texts(in_train)
in_train_s = en_tokenizer.texts_to_sequences(in_train)

In [103]:
hi_tokenizer.fit_on_texts(out_train)
out_train_s = hi_tokenizer.texts_to_sequences(out_train)

In [104]:
print(len(in_train_s) - len(out_train_s))
print(len(in_train) - len(out_train))

0
0


We are now ready for the word embedding phase of preprocessing where we can convert our texts to sequences of integers 

In [105]:
#Tokenize the texts and convert to sequences
en_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='', oov_token='<OOV>', lower=False)
en_tokenizer.fit_on_texts(train_data_en)
en_train_sequences = en_tokenizer.texts_to_sequences(train_data_en)

hi_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='', oov_token='<OOV>', lower=False)
hi_tokenizer.fit_on_texts(train_data_hi)
hi_train_sequences = hi_tokenizer.texts_to_sequences(train_data_hi)

In [106]:
len(en_train_sequences)

1659083

We now do the same for test and validation datasets.<br/>
After creating the embeddings for all data, we are going to fetch the final size of the vocabulary which can be fetched from the tokenizer object. 

In [107]:
en_tokenizer.fit_on_texts(test_data_en)
en_test_sequences = en_tokenizer.texts_to_sequences(test_data_en)
en_tokenizer.fit_on_texts(validation_data_en)
en_validation_sequences = en_tokenizer.texts_to_sequences(validation_data_en)

hi_tokenizer.fit_on_texts(test_data_hi)
hi_test_sequences = hi_tokenizer.texts_to_sequences(test_data_hi)
hi_tokenizer.fit_on_texts(validation_data_hi)
hi_validation_sequences = hi_tokenizer.texts_to_sequences(validation_data_hi)


english_vocab_size = len(en_tokenizer.word_index) + 1
hindi_vocab_size = len(en_tokenizer.word_index) + 1

print("Vocab size of english data: ", english_vocab_size)
print("Vocab size of hindi data: ", hindi_vocab_size)

Vocab size of english data:  385902
Vocab size of hindi data:  385902


A simple look at one of the embeddings will help us understand how the embedding was done by keras 

In [108]:
print(en_train_sequences[0])
print(train_data_en[0])

[144, 42, 414, 36, 7650, 44066]
give your application an accessibility workout


Now we pad all the sequences so that they are all of uniform length<br/>
Note that we are using English sentences as inputs to the encoder part of our transformer model and the hindi ones as both the outputs and inputs for the decoder of the transformer<br/>
The latter is because we need the translated output from the decoder and we need to train it with the target language sequences 

In [109]:
#Pad english sentences for using them as encoder inputs  
en_train_sequences = tf.keras.preprocessing.sequence.pad_sequences(en_train_sequences, maxlen=maxlen, padding='post')
en_test_sequences = tf.keras.preprocessing.sequence.pad_sequences(en_test_sequences, maxlen=maxlen, padding='post')
en_validation_sequences = tf.keras.preprocessing.sequence.pad_sequences(en_validation_sequences, maxlen=maxlen, padding='post')

#Pad hindi sentences for using them as decoder outputs and inputs 
decoder_inputs_train = []
decoder_outputs_train = []

# remove the last token in input and first token in the output. This is because, decoder should be trained such that for every input token
# the next token in sequence should be presented as the output. The training will be done accordingly.
for hi in hi_train_sequences:
  decoder_inputs_train.append(hi[:-1]) 
  decoder_outputs_train.append(hi[1:])

decoder_inputs_train = tf.keras.preprocessing.sequence.pad_sequences(decoder_inputs_train, maxlen=maxlen, padding='post')
decoder_outputs_train = tf.keras.preprocessing.sequence.pad_sequences(decoder_outputs_train, maxlen=maxlen, padding='post')


decoder_inputs_test = []
decoder_outputs_test = []

for hi in hi_test_sequences:
  decoder_inputs_test.append(hi[:-1])
  decoder_outputs_test.append(hi[1:])

decoder_inputs_test = tf.keras.preprocessing.sequence.pad_sequences(decoder_inputs_test, maxlen=maxlen, padding='post')
decoder_outputs_test = tf.keras.preprocessing.sequence.pad_sequences(decoder_outputs_test, maxlen=maxlen, padding='post')

decoder_inputs_validation = []
decoder_outputs_validation = []

for hi in hi_validation_sequences:
  decoder_inputs_validation.append(hi[:-1])
  decoder_outputs_validation.append(hi[1:])

decoder_inputs_validation = tf.keras.preprocessing.sequence.pad_sequences(decoder_inputs_validation, maxlen=maxlen, padding='post')
decoder_outputs_validation = tf.keras.preprocessing.sequence.pad_sequences(decoder_outputs_validation, maxlen=maxlen, padding='post')

In [110]:
print("English sentence: ", train_data_en[0])
print("Encoding: ", en_train_sequences[0])

English sentence:  give your application an accessibility workout
Encoding:  [  144    42   414    36  7650 44066     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0]


In [111]:
#Define LSTM model
d_model = 256    #dimension of the internal vector representations (h,c and embedding vectors)

#Encoder
inputs = tf.keras.layers.Input(shape=(None,))
x = tf.keras.layers.Embedding(english_vocab_size, d_model, mask_zero=True)(inputs)
_, state_h, state_c = tf.keras.layers.LSTM(d_model,activation='relu',return_state=True)(x)

#Decoder
targets = tf.keras.layers.Input(shape=(None,))
embedding_layer = tf.keras.layers.Embedding(hindi_vocab_size, d_model, mask_zero=True)
x = embedding_layer(targets)
decoder_lstm = tf.keras.layers.LSTM(d_model,activation='relu',return_sequences=True, return_state=True)
x,_,_ = decoder_lstm(x, initial_state=[state_h, state_c])
dense1 = tf.keras.layers.Dense(hindi_vocab_size, activation='softmax')
x = dense1(x)

model = tf.keras.models.Model(inputs=[inputs, targets],outputs=x)
model.summary()

loss = tf.keras.losses.SparseCategoricalCrossentropy()
model.compile(optimizer='rmsprop', loss=loss, metrics=['accuracy'])

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_9 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 input_10 (InputLayer)          [(None, None)]       0           []                               
                                                                                                  
 embedding_8 (Embedding)        (None, None, 256)    98790912    ['input_9[0][0]']                
                                                                                                  
 embedding_9 (Embedding)        (None, None, 256)    98790912    ['input_10[0][0]']               
                                                                                            

In [112]:
#Save model after each epoch
save_model_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='./drive/MyDrive/ml/models/en-hi.h5',
    monitor='val_accuracy',
    mode='max'
)

In [113]:
X_train = [en_train_sequences, decoder_inputs_train]
y_train = decoder_outputs_train

model.fit(train_data_en, train_data_hi, epochs=70, callbacks=[save_model_callback, tf.keras.callbacks.TerminateOnNaN()])

MemoryError: Unable to allocate 62.9 GiB for an array with shape (1659083,) and data type <U10181

In [ ]:
#Retrieve previously saved stuff
saved_model = tf.keras.models.load_model('./drive/MyDrive/ml/models/en-hi.h5')

saved_model.summary()

inputs = saved_model.get_layer('input_1').output
_,state_h,state_c = saved_model.get_layer('lstm').output
targets = saved_model.get_layer('input_2').output
embedding_layer = saved_model.get_layer('embedding_1')
decoder_lstm = saved_model.get_layer('lstm_1')
dense1 = saved_model.get_layer('dense')

OSError: No file or directory found at ./drive/MyDrive/ml/models/en-hi.h5

## 3. Model 

With the data preprocessing done, we should now proceed towards building the transformer which will perform the language translation task<br/>

First let us prepare the embedding layer 

In [ ]:
from torch import nn

class Embedder(nn.Module):
    def __init__(self, vocab_size, d_model):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, d_model)
    def forward(self, x):
        embedding = self.embed(x)
        return embedding


Next we add the positional encoder 

In [ ]:
import math
import torch 
from torch.autograd import Variable

class PositionalEncoder(nn.Module):
    def __init__(self, d_model, max_seq_len = 80):
        super().__init__()
        self.d_model = d_model
        
        # create constant 'pe' matrix with values dependant on 
        # pos and i
        pe = torch.zeros(max_seq_len, d_model)
        for pos in range(max_seq_len):
            for i in range(0, d_model, 2):
                pe[pos, i] = math.sin(pos / (10000 ** ((2 * i)/d_model)))
                pe[pos, i + 1] = math.cos(pos / (10000 ** ((2 * (i + 1))/d_model)))
                
        self.register_buffer('pe', pe)
 
    
    def forward(self, x):
        # make embeddings relatively larger
        x = x * math.sqrt(self.d_model)

        #add constant to embedding
        seq_len = x.size(0)
        
        x = x + Variable(self.pe[:seq_len], requires_grad=False)

        return x

Multi head attention 

In [ ]:
import torch.nn.functional as F

class MultiHeadAttention(nn.Module):
    def __init__(self, heads, d_model, dropout = 0.1):
        super().__init__()
        
        self.d_model = d_model
        self.d_k = d_model // heads
        self.h = heads
        
        self.q_linear = nn.Linear(d_model, d_model)
        self.v_linear = nn.Linear(d_model, d_model)
        self.k_linear = nn.Linear(d_model, d_model)
        self.dropout = nn.Dropout(dropout)
        self.out = nn.Linear(d_model, d_model)
    
    def forward(self, q, k, v, mask=None):
        
        batch_size = q.size(0)
        
        # perform linear operation and split into h heads
        
        k = self.k_linear(k).view(batch_size, -1, self.h, self.d_k)
        q = self.q_linear(q).view(batch_size, -1, self.h, self.d_k)
        v = self.v_linear(v).view(batch_size, -1, self.h, self.d_k)
        
        # transpose to get dimensions bs * h * sl * d_model

        k = k.transpose(1,2)
        q = q.transpose(1,2)
        v = v.transpose(1,2)


        # calculate attention using function we will define next
        scores = self.attention(q, k, v, self.d_k, mask, self.dropout)
        
        # concatenate heads and put through final linear layer
        concat = scores.transpose(1,2).contiguous().view(batch_size, -1, self.d_model)

        output = self.out(concat)
    
        return output
    
    def attention(self, q, k, v, d_k, mask=None, dropout=None):
        
        scores = torch.matmul(q, k.transpose(-2, -1)) /  math.sqrt(d_k)

        # if mask is not None:
        #     mask = mask.unsqueeze(1)
        #     scores = scores.masked_fill(mask == 0, -1e9)
        scores = F.softmax(scores, dim=-1)

        if dropout is not None:
            scores = dropout(scores)

        output = torch.matmul(scores, v)

        return output

Feed forward neural network

In [ ]:
class FeedForward(nn.Module):
    def __init__(self, d_model, d_ff=2048, dropout = 0.1):
        super().__init__() 
        # We set d_ff as a default to 2048
        self.linear_1 = nn.Linear(d_model, d_ff)
        self.dropout = nn.Dropout(dropout)
        self.linear_2 = nn.Linear(d_ff, d_model)
    def forward(self, x):
        x = self.dropout(F.relu(self.linear_1(x)))
        x = self.linear_2(x)
        return x

Various masks are created here 

<ul>
<li>Padding mask for English</li>
<li>Padding mask for Hindi</li>
<li>No peek mask for masking future tokens in order to avoid decoder from seeing the inputs early</li>
</ul>

In [ ]:
size = maxlen

def no_peek_mask():
    nopeek_mask = np.triu(np.ones((1, size, size)), k=1).astype('uint8')
    nopeek_mask = torch.from_numpy(nopeek_mask) == 0

    return nopeek_mask

def padding_mask(text):
    pad_mask = text != 0
    return pad_mask

def create_masks(src_text, target_text):
    return padding_mask(src_text), no_peek_mask() & padding_mask(target_text)


no_peek_mask().unsqueeze(1).shape

torch.Size([1, 1, 30, 30])

Normalization layer 

In [ ]:
class Norm(nn.Module):
    def __init__(self, d_model, eps = 1e-6):
        super().__init__()
    
        self.size = d_model
        # create two learnable parameters to calibrate normalisation
        self.alpha = nn.Parameter(torch.ones(self.size))
        self.bias = nn.Parameter(torch.zeros(self.size))
        self.eps = eps
    def forward(self, x):
        norm = self.alpha * (x - x.mean(dim=-1, keepdim=True))/(x.std(dim=-1, keepdim=True) + self.eps) + self.bias
        return norm

In [ ]:
import copy

# build an encoder layer with one multi-head attention layer and one feed-forward layer
class EncoderLayer(nn.Module):
    def __init__(self, d_model, heads, dropout = 0.1):
        super().__init__()
        self.norm_1 = Norm(d_model)
        self.norm_2 = Norm(d_model)
        self.attn = MultiHeadAttention(heads, d_model)
        self.ff = FeedForward(d_model)
        self.dropout_1 = nn.Dropout(dropout, inplace=True)
        self.dropout_2 = nn.Dropout(dropout, inplace=True)
        
    def forward(self, x, mask):
        x2 = self.norm_1(x)
        x = x + self.dropout_1(self.attn(x2,x2,x2,mask).squeeze(1))
        x2 = self.norm_2(x)
        x = x + self.dropout_2(self.ff(x2))
        return x
    
# build a decoder layer with two multi-head attention layers and one feed-forward layer
class DecoderLayer(nn.Module):
    def __init__(self, d_model, heads, dropout=0.1):
        super().__init__()
        self.norm_1 = Norm(d_model)
        self.norm_2 = Norm(d_model)
        self.norm_3 = Norm(d_model)
        
        self.dropout_1 = nn.Dropout(dropout, inplace=True)
        self.dropout_2 = nn.Dropout(dropout)
        self.dropout_3 = nn.Dropout(dropout)
        
        self.attn_1 = MultiHeadAttention(heads, d_model)
        self.attn_2 = MultiHeadAttention(heads, d_model)
        self.ff = FeedForward(d_model)

    def forward(self, x, e_outputs, src_mask, trg_mask):
            x2 = self.norm_1(x)
            x = x + self.dropout_1(self.attn_1(x2, x2, x2, trg_mask).squeeze(1))
            x2 = self.norm_2(x)
            x = x + self.dropout_2(self.attn_2(x2, e_outputs, e_outputs, src_mask).squeeze(1))
            x2 = self.norm_3(x)
            x = x + self.dropout_3(self.ff(x2))
            return x
# We can then build a convenient cloning function that can generate multiple layers:
def get_clones(module, N):
    return nn.ModuleList([copy.deepcopy(module) for i in range(N)])

In [ ]:
class Encoder(nn.Module):
    def __init__(self, vocab_size, d_model, N, heads):
        super().__init__()
        self.N = N
        self.embed = Embedder(vocab_size, d_model)
        self.pe = PositionalEncoder(d_model)
        self.layers = get_clones(EncoderLayer(d_model, heads), N)
        self.norm = Norm(d_model)
    def forward(self, src, mask):
        x = self.embed(src)
        x = self.pe(x)
        for i in range(self.N):
            x = self.layers[i](x, mask)
        return self.norm(x)
    
class Decoder(nn.Module):
    def __init__(self, vocab_size, d_model, N, heads):
        super().__init__()
        self.N = N
        self.embed = Embedder(vocab_size, d_model)
        self.pe = PositionalEncoder(d_model)
        self.layers = get_clones(DecoderLayer(d_model, heads), N)
        self.norm = Norm(d_model)
        self.softmax_layer = tf.keras.layers.Softmax()
    def forward(self, trg, e_outputs, src_mask, trg_mask):
        x = self.embed(trg)
        x = self.pe(x)
        for i in range(self.N):
            x = self.layers[i](x, e_outputs, src_mask, trg_mask)
        x =  self.norm(x)
        return x

In [ ]:
class Transformer(nn.Module):
    def __init__(self, src_vocab, trg_vocab, d_model, N, heads):
        super().__init__()
        self.encoder = Encoder(src_vocab, d_model, N, heads)
        self.decoder = Decoder(trg_vocab, d_model, N, heads)
        self.out = nn.Linear(d_model, trg_vocab)
        self.softmax = tf.keras.layers.Softmax()
    def forward(self, src, trg, src_mask, trg_mask):
        e_outputs = self.encoder.forward(src, src_mask)
        d_output = self.decoder.forward(trg, e_outputs, src_mask, trg_mask)
        output = self.out(d_output)
        return output

In [ ]:
d_model = 512
heads = 8
N = 2

#Save model after each epoch
checkpoint = tf.keras.callbacks.ModelCheckpoint("best_model1.hdf5", monitor='val_accuracy', verbose=1, save_best_only=True, mode='auto', save_weights_only=False)

model = Transformer(english_vocab_size, hindi_vocab_size, d_model, N, heads)

for p in model.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)
    
# this code is very important! It initialises the parameters with a
# range of values that stops the signal fading or getting too big.
# See this blog for a mathematical explanation.
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9, weight_decay=0.01)

In [ ]:
for p in model.named_parameters():
    print(p)

# Viewing the model parameters which would be subjected to the optimization 

('encoder.embed.embed.weight', Parameter containing:
tensor([[-7.7500e-03,  5.3169e-03, -3.6654e-03,  ..., -5.1208e-03,
         -4.3784e-03, -6.2394e-03],
        [ 1.1998e-11,  1.9891e-13,  1.6910e-11,  ..., -2.5530e-12,
          9.3223e-13, -8.7114e-11],
        [ 4.2669e-04, -2.4123e-05,  4.3228e-05,  ..., -2.9731e-04,
         -1.7009e-04, -3.0308e-04],
        ...,
        [-1.6315e-12, -2.7484e-12, -4.5370e-14,  ...,  1.8389e-11,
         -2.8895e-12, -2.8215e-12],
        [-5.2285e-12,  8.6875e-11, -8.5100e-12,  ..., -3.4739e-12,
          4.1022e-12, -1.8785e-12],
        [-5.2546e-11,  7.5752e-13, -1.2319e-11,  ...,  5.0035e-11,
          7.5875e-11,  2.0289e-12]], requires_grad=True))
('encoder.layers.0.norm_1.alpha', Parameter containing:
tensor([0.9610, 0.9612, 0.9611, 0.9611, 0.9611, 0.9612, 0.9612, 0.9611, 0.9611,
        0.9612, 0.9611, 0.9611, 0.9611, 0.9610, 0.9611, 0.9611, 0.9612, 0.9611,
        0.9612, 0.9611, 0.9611, 0.9611, 0.9611, 0.9611, 0.9612, 0.9611, 0.9610

In [ ]:
import time 

def train_model(epochs, print_every=10):
    
    model.train()
    
    start = time.time()
    temp = start
    
    total_loss = 0
    
    for epoch in range(epochs):
        for en_train_sequence, target_lang_input, target_lang_output in zip(en_train_sequences, decoder_inputs_train, decoder_outputs_train):
            source = torch.from_numpy(en_train_sequence)
            target_lang_input = torch.from_numpy(target_lang_input)
            target_lang_output = torch.from_numpy(target_lang_output)
            target_lang_output = F.one_hot(target_lang_output.to(torch.int64), num_classes=hindi_vocab_size)
            
            # create function to make masks using mask code above
            src_mask, trg_mask = create_masks(source, target_lang_input)
            
            preds = model(source, target_lang_input, src_mask, trg_mask)

            optimizer.zero_grad()

            loss = F.cross_entropy(preds, target_lang_output.to(torch.float64))
            loss.backward()
            optimizer.step()

            print(loss.data)
            
            total_loss += loss.data.item()

            if (epoch + 1) % print_every == 0:
                loss_avg = total_loss / print_every
                print("time = %dm, epoch %d, loss = %.3f, %ds per %d iters" % ((time.time() - start) // 60,
                epoch + 1, loss_avg, time.time() - temp, print_every))
                total_loss = 0
                temp = time.time()

train_model(10)

tensor(0.8993, dtype=torch.float64)
tensor(1.2894, dtype=torch.float64)
tensor(0.3899, dtype=torch.float64)
tensor(0.9041, dtype=torch.float64)
tensor(0.0018, dtype=torch.float64)
tensor(0.0018, dtype=torch.float64)
tensor(0.0019, dtype=torch.float64)
tensor(0.3902, dtype=torch.float64)
tensor(0.3901, dtype=torch.float64)
tensor(1.1186, dtype=torch.float64)
tensor(0.9532, dtype=torch.float64)
tensor(0.0019, dtype=torch.float64)
tensor(0.7793, dtype=torch.float64)
tensor(0.7786, dtype=torch.float64)
tensor(0.0020, dtype=torch.float64)
tensor(0.3855, dtype=torch.float64)
tensor(1.1649, dtype=torch.float64)
tensor(0.7790, dtype=torch.float64)
tensor(0.6552, dtype=torch.float64)
tensor(0.0020, dtype=torch.float64)
tensor(0.0019, dtype=torch.float64)
tensor(0.0019, dtype=torch.float64)
tensor(0.5078, dtype=torch.float64)
tensor(0.4434, dtype=torch.float64)
tensor(0.0018, dtype=torch.float64)
tensor(0.3904, dtype=torch.float64)
tensor(0.0017, dtype=torch.float64)
tensor(0.0017, dtype=torch.f

KeyboardInterrupt: 